In [0]:
from pyspark.sql import SparkSession, functions as f

#Reading Hospital A providers data
df_hosa=spark.read.parquet("/mnt/bronze/hosa/providers")

#Reading Hospital B providers data
df_hosb=spark.read.parquet("/mnt/bronze/hosb/providers")

#union two providers dataframes
df_merged = df_hosa.unionByName(df_hosb)
display(df_merged)

df_merged.createOrReplaceTempView("providers")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS silver.providers(
  ProviderID STRING,
  FirstName STRING,
  LastName STRING,
  Specialization STRING,
  DeptID STRING,
  NPI LONG,
  datasource STRING,
  is_quarantined BOOLEAN
) 
USING DELTA;

In [0]:
%sql
truncate table silver.providers

In [0]:
%sql
insert into silver.providers
SELECT
  ProviderID,
  FirstName,
  LastName,
  Specialization,
  DeptID,
  cast(NPI as INT) NPI,
  datasource,
    CASE
        WHEN ProviderID IS NULL OR DeptID IS NULL THEN TRUE 
        ELSE FALSE
    END AS is_quarantined
FROM providers 
